# Speckle Tracking

In [1]:
from pathlib import Path
# from project_heart.lv import LV
from project_heart.utils import set_jupyter_backend
from project_heart.enums import *
set_jupyter_backend("pythreejs")
import logging
import numpy as np
float_formatter = "{:.5f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

from project_heart.examples import get_lv_ideal
from project_heart.examples import get_lv_typeA

lv_ideal = get_lv_ideal(Path("../../_static/sample_files/ideal_linear_pressure_increase copy.xplt"))
lv_typeA = get_lv_typeA(
    Path("../../_static/sample_files/lv_typeA_hex.vtk"),
    Path("../../_static/sample_files/sample_displacement_lv_typeA_hex_states_linear_press_incr.pbz2")
    )

sample_spk_typeA_endo = lv_typeA.get_speckles(spk_name="SAMPLE", spk_group="endo", spk_collection="SAMPLE")[0]
sample_spk_typeA_epi = lv_typeA.get_speckles(spk_name="SAMPLE", spk_group="epi", spk_collection="SAMPLE")[0]

## What are Speckles?

Speckles ...

In [2]:
normal = [1,0,0]
n_subsets = 2
_ = lv_typeA.create_speckles(
            collection="SAMPLE",
            group="epi",
            name="SAMPLE-LONG",
            from_nodeset=LV_SURFS.EPI,
            exclude_nodeset=[LV_SURFS.BASE, LV_SURFS.ENDO],
            d=2.5,
            k=0.5,
            normal_to=normal,
            n_subsets=n_subsets,
            subsets_criteria="z2",
            cluster_criteria="angles3",
            n_clusters=8,
            t=0.0,
            kmin=-1,
            kmax=0.85,
            log_level=logging.DEBUG,
        )

sample_spk_typeA = lv_typeA.get_speckles(spk_name="SAMPLE-LONG", spk_group="epi", spk_collection="SAMPLE")

spk = sample_spk_typeA
plotter = lv_typeA.plot_speckles(spk, 
                       cmap="hot",
                    #    color="blue",
                       point_size=300,
                       add_k_centers=True, 
                       k_centers_as_line=True, 
                       k_bins=True,
                       )

INFO:create_speckles:Speckle: name: SAMPLE-LONG, group: epi, collection: SAMPLE
DEBUG:create_speckles:Using nodes from nodeset LV_SURFS.EPI
DEBUG:create_speckles:Excluding nodes from nodeset [<LV_SURFS.BASE: 23>, <LV_SURFS.ENDO: 1>]
DEBUG:create_speckles:Normal: [1.00000 0.00000 0.00000]
DEBUG:create_speckles:long_line: [[1.68662 -0.54943 4.05614]
 [0.28934 3.90746 83.33411]]
DEBUG:create_speckles:ref p (spk center): [0.98798 1.67901 43.69512]
DEBUG:create_speckles:pts close to plane: 655
DEBUG:create_speckles:Adjusting for geometry aligned with normal. kmax: 0.85
DEBUG:create_speckles:New spk center and pts found after adjustment.
DEBUG:create_speckles:ref p (spk center): [0.49893 3.23893 71.44241]
DEBUG:create_speckles:pts close to plane: 561
DEBUG:create_speckles:pts: 561
DEBUG:create_speckles:Adding multiple subsets: 2 -> [0, 1]
DEBUG:_subdivide_speckles:Subdividing Speckles into '2' buckets based on subsets_criteria: 'z2'.
DEBUG:_subdivide_speckles:Number of buckets: 2.
DEBUG:_sub

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(position=(50000.000000000015, 70710.6…

In [4]:
from project_heart.utils.vector_utils import *
normal = [1,0,0]
n_subsets = 2
n_cluster = 8
_ = lv_typeA.create_speckles(
            collection="SAMPLE",
            group="epi",
            name="SAMPLE-LONG",
            from_nodeset=LV_SURFS.EPI,
            exclude_nodeset=[LV_SURFS.BASE, LV_SURFS.ENDO],
            d=2.5,
            k=0.5,
            normal_to=normal,
            n_subsets=n_subsets,
            subsets_criteria="z2",
            # cluster_criteria="z2",
            n_clusters=0,
            t=0.0,
            kmin=-1,
            kmax=0.85,
            log_level=logging.WARN,
        )

sample_spk_typeA = lv_typeA.get_speckles(spk_name="SAMPLE-LONG", spk_group="epi", spk_collection="SAMPLE")
for i in range(n_subsets):
    
    spk = sample_spk_typeA[i]
    ref_center = spk.center
    n_subsets = 4
    pts = lv_typeA.points(mask=spk.ids)

    import logging
    logger = logging.getLogger("test")
    logger.setLevel(logging.DEBUG)
    logger.debug("i: {}.".format(i))
    
    # project points onto a single plane based on spk normal
    logger.debug("pts.shape: {}.".format(pts.shape))
    logger.debug("pts[:5]: \n{}.".format(pts[:5]))
    plane_d = calc_plane_d(normal, ref_center)
    logger.debug("plane_d: {}.".format(plane_d))
    ppts = project_pts_onto_plane(pts, normal, plane_d)
    p_center = project_pts_onto_plane(ref_center, normal, plane_d)[0]
    logger.debug("p_center: {}.".format(p_center))
    # compute angles between a reference vector and other vectos
    vecs = ppts - p_center 
    ref_max_pt = np.argmax(np.linalg.norm(vecs, axis=1))
    ref_min_pt = np.argmin(np.linalg.norm(vecs, axis=1))
    logger.debug("ref_max_pt: {}.".format(ref_max_pt))
    logger.debug("ref_max_pt: {}.".format(ref_min_pt))
    ref_max_vec = vecs[ref_max_pt]
    ref_min_vec = vecs[ref_min_pt]
    logger.debug("ref_max_vec: {}.".format(ref_max_vec))
    angles = angle_between(vecs, ref_max_vec, check_orientation=False)
    min_a, max_a = np.min(angles), np.max(angles)
    logger.debug("min_a: {}.".format(min_a))
    logger.debug("max_a: {}.".format(max_a))
    # create bins
    bins = np.digitize(
        angles, np.linspace(min_a*0.999, max_a*1.001, n_cluster+1))
    # sort bins in 'ascending' order based on first bin 
    # this is useful to keep consistency between speckles
    angle_between_refs = angle_between(ref_max_vec, ref_min_vec, 
                                       check_orientation=True, zaxis=normal)   
    logger.debug("angle_between_refs: {}.".format(np.degrees(angle_between_refs)))
    if angle_between_refs > np.radians(180):
        logger.debug("reversing bins: {} > 180.".format(np.degrees(angle_between_refs)))
        uvals = np.unique(bins)
        logger.debug("uvals: {}.".format(uvals))
        new_bins = np.zeros(len(bins), dtype=np.int64)
        for i, u in enumerate(uvals[::-1]):
            new_bins[bins==u] = i
        bins = new_bins + 1
        
    plotter = lv_typeA.plot_speckles(spk, 
                        cmap=None,
                        color="blue",
                        point_size=100,
                        add_k_centers=False, 
                        k_centers_as_line=True, 
                        k_bins=False,
                        re = True
                        )
    plotter.add_lines(np.vstack((p_center, pts[ref_max_pt])), color="blue")
    plotter.add_lines(np.vstack((p_center, pts[ref_min_pt])), color="green")
    plotter.add_points(p_center, color="green", point_size=240)
    # plotter.add_points(ppts, color="red", point_size=200)
    plotter.add_points(pts, scalars=bins-1, categories=True, cmap="hot", point_size=240)
    plotter.show(window_size=(600,400))

DEBUG:test:i: 0.
DEBUG:test:pts.shape: (252, 3).
DEBUG:test:pts[:5]: 
[[0.99841 -33.61353 39.83204]
 [0.53665 -32.94615 37.49577]
 [0.03292 -32.25283 35.03556]
 [-1.48766 -37.31217 52.96571]
 [-0.53072 -38.19690 68.99477]].
DEBUG:test:plane_d: -0.9879793524742126.
DEBUG:test:p_center: [0.98798 1.67901 43.69512].
DEBUG:test:ref_max_pt: 77.
DEBUG:test:ref_max_pt: 212.
DEBUG:test:ref_max_vec: [0.00000 -39.86261 26.61626].
DEBUG:test:min_a: 2.8889043806810274e-06.
DEBUG:test:max_a: 2.1564495312360212.
DEBUG:test:angle_between_refs: 56.57087845878773.


Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(position=(50000.000000000015, 70710.6…

DEBUG:test:i: 1.
DEBUG:test:pts.shape: (309, 3).
DEBUG:test:pts[:5]: 
[[2.08964 31.88098 65.58775]
 [2.21676 31.82784 63.47303]
 [0.04540 32.39156 55.31783]
 [2.71947 31.94998 55.33662]
 [2.77303 32.13464 53.40599]].
DEBUG:test:plane_d: -0.9879793524742126.
DEBUG:test:p_center: [0.98798 1.67901 43.69512].
DEBUG:test:ref_max_pt: 186.
DEBUG:test:ref_max_pt: 122.
DEBUG:test:ref_max_vec: [0.00000 0.81769 -43.50979].
DEBUG:test:min_a: 3.031791296491834e-06.
DEBUG:test:max_a: 2.289417489803035.
DEBUG:test:angle_between_refs: 88.08945446815837.


Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(position=(50000.000000000015, 70710.6…

In [ ]:
sample_spk_typeA

In [ ]:
plotter = lv_typeA.plot_speckles(sample_spk_typeA, 
                       cmap="hot",
                    #    color="blue",
                       point_size=300,
                       add_k_centers=True, 
                       k_centers_as_line=True, 
                       k_bins=True,
                       )

## Sample Speckles

### Longitudinal Speckles

In [ ]:
# lv_ideal.plot_speckles(endo_long[1], add_centers=True, cmap="tab10", categories=True)

In [ ]:
# lv_ideal.plot_speckles(epi_long[0], cmap="jet", add_k_centers=True, k_centers_as_line=True, k_bins=True, categories=True)

### Circumferential Speckles

In [ ]:
# lv_ideal.plot_speckles(endo_circ[7], add_centers=True, cmap="tab10", categories=True)

In [ ]:
# lv_ideal.plot_speckles(epi_circ[7], cmap="jet", add_k_centers=True, k_centers_as_line=True, k_bins=True, categories=True)